In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import operator
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk_stopwords= set(stopwords.words('english'))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
df= pd.read_json('/content/drive/MyDrive/amazon_data_scrapping/Data files/product3_reviews_details.json', orient='index')
df.head()

In [ ]:
df['reviewText']

In [7]:
df['reviewDate']= df['reviewDate'].apply(lambda date:re.sub('Reviewed in the United States on ',"", date).strip())
df['reviewRating']= df['reviewRating'].apply(lambda rating: re.sub(' out of 5 stars',"", rating).strip()) 
df['reviewRating']= df.reviewRating.astype('float')
df['reviewClass']=df.apply(lambda row: 1 if row.reviewRating>=4.0 else 0, axis=1)
# df['reviewText']= df['reviewTitle'] + '. '+ df['reviewText']
# df['reviewText']= df['reviewText'].apply(lambda review: review.lower())
df['reviewText'] = df['reviewText'].apply(lambda review:re.sub("\n", ' ', review))

#shortforms
df['reviewText']= df['reviewText'].apply(lambda review: re.sub('bc|b/c', 'because', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("wasn’t", 'was not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("i’m", 'i am', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("i'm", 'i am', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("doesn’t", 'does not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("doesn't", 'does not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("didn’t", 'did not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("won't", 'will not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("won’t", 'will not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("don’t", 'do not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("can't", 'can not', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("it's", 'it is', review))
df['reviewText']= df['reviewText'].apply(lambda review: re.sub("it’s", 'it is', review))
# df['reviewText']= df['reviewText'].apply(lambda review: re.sub("(\\d|\\W)+", ' ', review))
# df['reviewText']= df['reviewText'].apply(lambda review: re.sub("(\\d)+", ' ', review))

In [5]:
# def get_sentiment(score):
#   if score>= 0:
#     return 1
#   else:
#     return 0

# sia_object= SentimentIntensityAnalyzer()
# df['compound']= df['reviewText'].apply(lambda review: sia_object.polarity_scores(review)['compound'])
# df['sentiment']= df['compound'].apply(lambda score: get_sentiment(score))

In [ ]:
#rating 2.0 has positive sentiment
df.head()

In [ ]:
#155 positive reviews
df[df['sentiment']==1].shape

(2743, 6)

In [ ]:
#5 negative reviews
df[df['sentiment']==0].shape

(727, 6)

In [8]:
# reviews=df[df['sentiment']==1]['reviewText']
reviews=df[df['reviewClass']==1]['reviewText']
positive_reviews=''
for review in reviews:
  positive_reviews= positive_reviews+" "+review

In [9]:
# reviews=df[df['sentiment']==0]['reviewText']
reviews=df[df['reviewClass']==0]['reviewText']
negative_reviews=''
for review in reviews:
  negative_reviews= negative_reviews+" "+review

In [11]:
sentences= sent_tokenize(positive_reviews)
len(sentences)

4939

In [12]:
sentences[:5]

[" It looks really nice, and having the auto brew is awesome, but the basket area is a bit big and so the basket does not have a snug fit and moves around when in place making me worry that the coffee flow valve on the basket isn't open all the way causing the basket to overflow.",
 "It hasn't happened yet and perhaps it will not.",
 'Only time will tell.',
 'Not a bad coffee pot for $25.',
 'The digital display does look good and easy to see.']

In [13]:
cleaned_sentences= pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
cleaned_sentences= [sentence.lower() for sentence in cleaned_sentences]
def remove_stopwords(sentence):
    new_sentence = " ".join([word for word in sentence if word not in nltk_stopwords])
    return new_sentence
cleaned_sentences = [remove_stopwords(sentence.split()) for sentence in cleaned_sentences]
# cleaned_sentences[:5]

In [14]:
cleaned_sentences[:5]

['looks really nice auto brew awesome basket area bit big basket snug fit moves around place making worry coffee flow valve basket open way causing basket overflow',
 'happened yet perhaps',
 'time tell',
 'bad coffee pot',
 'digital display look good easy see']

In [ ]:
#why have i used 100d file
#pickle the word_embeddings dictionary
word_embeddings={}
file= open('/content/drive/MyDrive/amazon_data_scrapping/Colab notebooks/glove/glove.6B.100d.txt')
for line in file:
  embeddings= line.split()
  word= embeddings[0]
  weights = np.asarray(embeddings[1:], dtype='float32')
  word_embeddings[word] = weights
file.close()

In [ ]:
sentence_vectors=[]
for sentence in cleaned_sentences:
  if len(sentence)!=0:
    vector= sum([word_embeddings.get(word, np.zeros((100,))) for word in sentence.split()])/(len(sentence.split())+0.001)
  else:
    vector= np.zeros((100,))
  sentence_vectors.append(vector)

In [ ]:
similarity_matrix= np.zeros([len(sentences), len(sentences)])
similarity_matrix.shape

(6236, 6236)

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      similarity_matrix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100)) [0,0]

In [ ]:
import networkx as nx
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
ranked_sentences[:2]

[(0.006714905872105773,
  'The little wooden stands that come with them allow you to elevate the plants, giving you versatility in design, and I loved the way that looked.'),
 (0.00671192733533576,
  'So cute and a great price and great quality It’s beautiful and feels hand made.')]

In [ ]:
summary=''
for i in range(10):
  summary= summary+ ''+ ranked_sentences[i][1]
summary

'The little wooden stands that come with them allow you to elevate the plants, giving you versatility in design, and I loved the way that looked.So cute and a great price and great quality It’s beautiful and feels hand made.Pot is great stand is great plugs suck they do not work they give one standard plug for every size pot you buy they do not work,cheap,leak!!!I actually think it looks even nicer in person than in the photo.. Light weight and beautiful pot.Stand is like others at this price point, sturdy but a tiny bit off level.I love it and it makes the perfect addition to my room.One of the stands had the wrong piece so I could not use it.I’ve been looking for a new pot to save a struggling plant and this one was perfect!They each come with 2 little soft plastic plugs, so you would probably want to add a dish underneath to catch water if you leave it unplugged.I spray painted the wooden feet black because it goes better with my decor, but that was super easy and it is absolutely p

In [ ]:
#change to camelcase :Done
#apply spell check
#clean up extra spaces and periods
